<a href="https://colab.research.google.com/github/YKefasu/Colab_Notebook-Fooocus/blob/main/Yoshia's_Notebook_Fooocus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown #STEP 1️⃣: Persiapan
import os
import time
from IPython.display import clear_output, display, HTML
from IPython.display import clear_output
from datetime import timedelta
from subprocess import getoutput
from google.colab import drive
from google.colab.output import eval_js

def check_gpu():
    print("\033[96m")  # Cyan text
    print('⌚ Checking GPU...', end='')
    output = getoutput('nvidia-smi --query-gpu=gpu_name --format=csv')
    if "name" in output:
        gpu_name = output[5:]
        print('\r✅ Current GPU:', gpu_name, flush=True)
    else:
        print('\r\033[91m❎ ERROR: No GPU detected. Lakukan Step dibawah ini.\n', flush=True)
        # Additional code for displaying an image and message
        display(HTML("<img src='https://i.ibb.co/HC9KH17/NVIDIA-Share-23-01-02-173037.png' width='800px'/>"))
        print('\033[91m\nJika tertulis "Cannot connect to GPU backend", berarti telah mencapai batas pemakaian. Silahakn istirahat untuk hari ini...')
        display(HTML("<center><img src='https://media.tenor.com/RY9NX67klacAAAAi/sad-cute.gif' width='272px'/></center>"))
        time.sleep(5)
        from google.colab import runtime
        runtime.unassign()

def clone_FooocusMRE():
    %cd /content
    !git clone https://github.com/MoonRide303/Fooocus-MRE
    clear_output()

def install_jemalloc():
    %cd /content/automatic/
    !apt -y update -qq
    !apt -y install libjemalloc-dev
    %env LD_PRELOAD=/usr/lib/x86_64-linux-gnu/libjemalloc.so.2
    clear_output()

def install_req():
    %cd /content/Fooocus-MRE/
    !pip install torchsde==0.2.5 einops==0.4.1 transformers==4.30.2 safetensors==0.3.1 accelerate==0.21.0
    !pip install pytorch_lightning==1.9.4 omegaconf==2.2.3 gradio==3.39.0 xformers==0.0.20 triton==2.0.0 pygit2==1.12.2
    !pip install --use-feature=fast-deps -r requirements_versions.txt
    clear_output()

def download_models():
    !apt -y install -qq aria2
    clear_output()
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd_xl_base_1.0/resolve/main/sd_xl_base_1.0_0.9vae.safetensors -d /content/Fooocus-MRE/models/checkpoints -o sd_xl_base_1.0_0.9vae.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd_xl_refiner_1.0/resolve/main/sd_xl_refiner_1.0_0.9vae.safetensors -d /content/Fooocus-MRE/models/checkpoints -o sd_xl_refiner_1.0_0.9vae.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_offset_example-lora_1.0.safetensors -d /content/Fooocus-MRE/models/loras -o sd_xl_offset_example-lora_1.0.safetensors
    clear_output()

# Check GPU
start_colab = int(time.time()) - 5
check_gpu()

# Check GPU
clone_FooocusMRE()

#Install jemalloc
install_jemalloc()

#Install Requirement
install_req()

#Download Models
download_models()

from google.colab import drive
drive.mount('/content/drive')

clear_output()
print("✅ Done", "success")

# STEP 2️⃣: Perlengkapan

In [ ]:
#@markdown #Optional🧩: Models Downloader
from IPython.display import clear_output
import os
import requests
from urllib.parse import unquote

def download_model(Type, DirectLink_URL):
    if Type == "None" or not DirectLink_URL:
        print("Skipping download as no valid Type or DirectLink_URL provided.")
        return

    output_path = "/content/Fooocus-MRE/models/"

    if Type == "Checkpoint":
        output_path += "checkpoints/"
    elif Type == "LoRA":
        output_path += "Loras/"
    elif Type == "Textual Inversion":
        output_path += "embeddings/"
    else:
        print("Invalid type specified.")
        return

    # Create the directories if they don't exist
    os.makedirs(output_path, exist_ok=True)

    urls = [url.strip() for url in DirectLink_URL.split(",")]

    for url in urls:
        response = requests.get(url, stream=True)

        if response.status_code == 200:
            content_disposition = response.headers.get('content-disposition')
            if content_disposition:
                filename = unquote(content_disposition.split('filename=')[1])
            else:
                filename = unquote(url.split("/")[-1])  # Extracting the filename from the URL

            # Remove double quotes and semicolons from the filename
            filename = filename.replace('"', '').replace(';', '')

            filename = os.path.join(output_path, filename)  # Modify the filename to include the output path

            if not os.path.exists(filename):
                print("Downloading file:", filename)
                chunk_size = 5242880  # 5 MB
                with open(filename, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=chunk_size):
                        if chunk:
                            f.write(chunk)
                print("File downloaded successfully.")
            else:
                print("File already exists:", filename)
        else:
            print("Failed to download the file:", url)

    # Clear the output to keep the notebook clean
    clear_output()

    # Print the success message
    print("✅ Done", "success")

# Call the function with the first set of parameters
Type1 = "Checkpoint"  # @param ["Checkpoint", "LoRA", "Textual Inversion"]
DirectLink_URL1 = ""  # @param {'type': 'string'}
download_model(Type1, DirectLink_URL1)

# Call the function with the second set of parameters
Type2 = "None"  # @param ["None", "Checkpoint", "LoRA", "Textual Inversion"]
DirectLink_URL2 = ""  # @param {'type': 'string'}
download_model(Type2, DirectLink_URL2)

In [ ]:
#@markdown #STEP 3️⃣: START
%cd /content/Fooocus-MRE

!git pull
!python launch.py --share

In [ ]:
#@markdown #STEP 4️⃣: Copy Output Folder/Images to GoogleDrive
import os
import shutil

# Define paths
source_folder = '/content/Fooocus-MRE/outputs'
destination_folder = '/content/drive/MyDrive/Fooocus-IMG-Output/'

# Check if the source folder exists
if os.path.exists(source_folder):
    # Create the destination folder if it doesn't exist
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Recursively copy files from source subdirectories to destination
    for root, dirs, files in os.walk(source_folder):
        for filename in files:
            source_file_path = os.path.join(root, filename)
            rel_path = os.path.relpath(source_file_path, source_folder)
            destination_file_path = os.path.join(destination_folder, rel_path)

            # Create subdirectories in the destination if they don't exist
            destination_dir = os.path.dirname(destination_file_path)
            os.makedirs(destination_dir, exist_ok=True)

            # Check if the file is "log.html"
            if filename == "log.html":
                shutil.copy(source_file_path, destination_file_path)
                print(f"✅ File 'log.html' copied (overwritten).")
            else:
                # Check if the file already exists in the destination folder
                if not os.path.exists(destination_file_path):
                    shutil.copy(source_file_path, destination_file_path)
                    print(f"✅ File '{filename}' copied.")
                else:
                    print(f"File '{filename}' already exists. Skipping.")

    print("✅ DONE Copy operation completed")
else:
    print("Source folder not available.")